In [ ]:
import nest_asyncio
import asyncio
import logging
from telethon import TelegramClient, events
from openai import OpenAI

# تفعيل nest_asyncio لتجنب تعارض الحلقات التزامنية في Jupyter أو Colab
nest_asyncio.apply()

# تعريف المعلومات اللازمة
api_id = "<your_api_id>"
api_hash = "<your_api_hash>"
bot_token = "<your_bot_token>"

# تهيئة العميل
client = TelegramClient('bot', api_id, api_hash)
ai_client = OpenAI()

# تعريف الدالة الرئيسية
async def main():
    # بدء العميل
    await client.start(bot_token=bot_token)

    # معالج الأمر /start
    @client.on(events.NewMessage(pattern='/start'))
    async def start_handler(event):
        await event.respond("Hello! I am Hackers Realm AI Bot. How can I assist you today?")
        logging.info(f'Start command received from {event.sender_id}')

    # معالج الأمر /info
    @client.on(events.NewMessage(pattern='/info'))
    async def info_handler(event):
        await event.respond("This AI Chatbot is created in Python with OpenAI API.")
        logging.info(f'Info command received from {event.sender_id}')

    # معالج الأمر /help
    @client.on(events.NewMessage(pattern='/help'))
    async def help_handler(event):
        help_text = (
            "Here are the commands you can use:\n"
            "/start - Start the bot\n"
            "/help - Get Help Information\n"
            "/info - Get Information about the Bot\n"
        )
        await event.respond(help_text)
        logging.info(f"Help command received from {event.sender_id}")

    # معالج الاستجابة للكلمات الرئيسية
    @client.on(events.NewMessage)
    async def keyword_responder(event):
        message = event.text.lower()
        if message in ['/start', '/help', '/info']:
            return

        # استجابة من العميل AI
        response = ai_client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {'role': 'user', "content": message}
            ],
            max_tokens=128
        )

        # الحصول على المحتوى من الاستجابة
        response = response.choices[0].message.content
        if response:
            await event.respond(response)
        logging.info(f"Message received from {event.sender_id}: {event.text}")

    # تشغيل العميل حتى يتم قطع الاتصال
    await client.run_until_disconnected()

# تشغيل الدالة الرئيسية
await main()
